In [1]:
"""
================================================================================
Projeto: Coleta e Tratamento de Dados de Aerogeradores
Autor: Elias Martini
Data de Criação: 26/08/2025
Última Atualização: 26/08/2025
Descrição: 
  Script Python para coletar dados de aerogeradores a partir da API SIGEL 
  da ANEEL, realizar tratamento de inconsistências, e gerar arquivo CSV 
  com os dados tratados.
Versão: 1.0
================================================================================
"""

import requests
import geopandas as gpd
import pandas as pd

def FetchAllIds(url: str):
  """
  Consulta a API do SIGEL e retorna todos os ids de objetos.
  """
  params = {
    "f": "json",
    "where": "1=1",
    "returnIdsOnly": "true"
  }
  result = requests.get(url, params=params).json()
  ids = result["objectIds"]
  return ids

def FetchObjectsBatch(url: str, object_ids: list):
  """
  A partir de uma lista de ids, consulta a API do SIGEL e retorna os objetos correspondentes.
  """
  where_condition = f"OBJECTID IN ({','.join(map(str, object_ids))})"
  params = {
    "f": "geojson",
    "where": where_condition,
    "outFields": "*",
    "returnGeometry": "true"
  }
  result = requests.get(url, params=params)
  result.raise_for_status()
  return gpd.read_file(result.text)

def CorrectNominalPower(power: float):
  """
  Corrige valores de potência, considerando que valores coerentes 
  para aerogeradores não são maiores do que 100 MW ou menores do que 0,05 MW.
  """
  if power > 100:
    return power / 1000
  elif power < 0.05:
    return power * 1000
  else:
    return power

# ============================================
# Main
# ============================================

url = "https://sigel.aneel.gov.br/arcgis/rest/services/PORTAL/WFS/MapServer/0/query"

# Query de todos os Ids
object_ids = FetchAllIds(url)
print(f"Total de objetos: {len(object_ids)}")

# Query dos objetos em batchs de 1000 (limite da API)
batch_size = 1000
gdfs = []
for i in range(0, len(object_ids), batch_size):
  batch_ids = object_ids[i:i+batch_size]
  gdf_batch = FetchObjectsBatch(url, batch_ids)
  gdfs.append(gdf_batch)
  print(f"Lote {i//batch_size+1}: {len(gdf_batch)} registros")

# Empilha geoDataFrame final
gdf_total = pd.concat(gdfs, ignore_index=True)
print(f"GeoDataFrame final: {len(gdf_total)} registros")

# Cria colunas de lat/lon
gdf_total["longitude"] = gdf_total.geometry.x
gdf_total["latitude"] = gdf_total.geometry.y

# Converte coluna de timestamp (ms) em data
gdf_total["DATA_ATUALIZACAO_FORMATADA"] = (
  pd.to_datetime(gdf_total["DATA_ATUALIZACAO"], unit="ms", utc=True).dt.tz_convert("America/Sao_Paulo")
)

/Users/eliasmartini/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Total de objetos: 23522
Lote 1: 1000 registros
Lote 2: 1000 registros
Lote 3: 1000 registros
Lote 4: 1000 registros
Lote 5: 1000 registros
Lote 6: 1000 registros
Lote 7: 1000 registros
Lote 8: 1000 registros
Lote 9: 1000 registros
Lote 10: 1000 registros
Lote 11: 1000 registros
Lote 12: 1000 registros
Lote 13: 1000 registros
Lote 14: 1000 registros
Lote 15: 1000 registros
Lote 16: 1000 registros
Lote 17: 1000 registros
Lote 18: 1000 registros
Lote 19: 1000 registros
Lote 20: 1000 registros
Lote 21: 1000 registros
Lote 22: 1000 registros
Lote 23: 1000 registros
Lote 24: 522 registros
GeoDataFrame final: 23522 registros


## Verificação dos tipos das colunas

In [12]:
gdf_total.dtypes

POT_MW                                                  float64
ALT_TOTAL                                               float64
ALT_TORRE                                               float64
DIAM_ROTOR                                              float64
DATA_ATUALIZACAO                                          int64
EOL_VERSAO_ID                                           float64
NOME_EOL                                                 object
DEN_AEG                                                  object
X                                                       float64
Y                                                       float64
VERSAO                                                   object
DATUM_EMP                                                object
OPERACAO                                                 object
FUSO_AG                                                  object
PROPRIETARIO                                             object
ORIGEM                                  

## Contagem de valores vazios

In [2]:
gdf_total.isna().sum()

POT_MW                            0
ALT_TOTAL                         0
ALT_TORRE                         4
DIAM_ROTOR                        0
DATA_ATUALIZACAO                  0
EOL_VERSAO_ID                     9
NOME_EOL                          0
DEN_AEG                           0
X                                 0
Y                                 0
VERSAO                            0
DATUM_EMP                         0
OPERACAO                        206
FUSO_AG                           0
PROPRIETARIO                      9
ORIGEM                        23522
OBJECTID                          0
UF                                9
CEG                               9
geometry                          0
longitude                         0
latitude                          0
DATA_ATUALIZACAO_FORMATADA        0
dtype: int64

## Contagem da coluna OPERACAO

In [3]:
gdf_total["OPERACAO"].value_counts(dropna=False)

OPERACAO
Não     12878
Sim     10421
None      206
1          17
Name: count, dtype: int64

## Verificação de registros duplicados

In [8]:
gdf_total[gdf_total.duplicated(subset=["NOME_EOL", "DEN_AEG", "latitude", "longitude"], keep=False)]

,POT_MW,ALT_TOTAL,ALT_TORRE,DIAM_ROTOR,DATA_ATUALIZACAO,EOL_VERSAO_ID,NOME_EOL,DEN_AEG,X,Y,...,FUSO_AG,PROPRIETARIO,ORIGEM,OBJECTID,UF,CEG,geometry,longitude,latitude,DATA_ATUALIZACAO_FORMATADA
14450,4.5,188.0,110.0,155.0,1686579219000,32804.0,Aura Caetité 03,ACA-02-08,759387.0,8480672.0,...,UTM 23 S (MER -45),Tanque Novo VI Energias Renováveis S.A.,None,14451,BA,EOLCVBA032804-9-01,POINT (-42.60145 -13.73156),-42.601448,-13.731562,2023-06-12 11:13:39-03:00
14451,4.5,188.0,110.0,155.0,1686579219000,32804.0,Aura Caetité 03,ACA-02-07,759560.0,8480930.0,...,UTM 23 S (MER -45),Tanque Novo VI Energias Renováveis S.A.,None,14452,BA,EOLCVBA032804-9-01,POINT (-42.59987 -13.72922),-42.599873,-13.729215,2023-06-12 11:13:39-03:00
14452,4.5,188.0,110.0,155.0,1686579219000,32804.0,Aura Caetité 03,ACA-02-06,759624.0,8481301.0,...,UTM 23 S (MER -45),Tanque Novo VI Energias Renováveis S.A.,None,14453,BA,EOLCVBA032804-9-01,POINT (-42.59932 -13.72586),-42.599316,-13.725858,2023-06-12 11:13:39-03:00
14453,4.5,188.0,110.0,155.0,1686579219000,32804.0,Aura Caetité 03,ACA-02-05,760140.0,8481369.0,...,UTM 23 S (MER -45),Tanque Novo VI Energias Renováveis S.A.,None,14454,BA,EOLCVBA032804-9-01,POINT (-42.59455 -13.7252),-42.594554,-13.725197,2023-06-12 11:13:39-03:00
14454,4.5,188.0,110.0,155.0,1686579219000,32804.0,Aura Caetité 03,ACA-02-04,760152.0,8481696.0,...,UTM 23 S (MER -45),Tanque Novo VI Energias Renováveis S.A.,None,14455,BA,EOLCVBA032804-9-01,POINT (-42.59447 -13.72224),-42.594473,-13.722242,2023-06-12 11:13:39-03:00
14455,4.5,188.0,110.0,155.0,1686579219000,32804.0,Aura Caetité 03,ACA-02-03,760180.0,8482017.0,...,UTM 23 S (MER -45),Tanque Novo VI Energias Renováveis S.A.,None,14456,BA,EOLCVBA032804-9-01,POINT (-42.59424 -13.71934),-42.594244,-13.719339,2023-06-12 11:13:39-03:00
14456,4.5,188.0,110.0,155.0,1686579219000,32804.0,Aura Caetité 03,ACA-02-02,759853.0,8482739.0,...,UTM 23 S (MER -45),Tanque Novo VI Energias Renováveis S.A.,None,14457,BA,EOLCVBA032804-9-01,POINT (-42.59733 -13.71285),-42.597332,-13.712846,2023-06-12 11:13:39-03:00
14457,4.5,188.0,110.0,155.0,1686579219000,32804.0,Aura Caetité 03,ACA-02-01,759558.0,8483398.0,...,UTM 23 S (MER -45),Tanque Novo VI Energias Renováveis S.A.,None,14458,BA,EOLCVBA032804-9-01,POINT (-42.60012 -13.70692),-42.600118,-13.706919,2023-06-12 11:13:39-03:00
14458,4.5,188.0,110.0,155.0,1686580359000,32804.0,Aura Caetité 03,ACA-02-08,759387.0,8480672.0,...,UTM 23 S (MER -45),Tanque Novo VI Energias Renováveis S.A.,None,14459,BA,EOLCVBA032804-9-01,POINT (-42.60145 -13.73156),-42.601448,-13.731562,2023-06-12 11:32:39-03:00
14459,4.5,188.0,110.0,155.0,1686580359000,32804.0,Aura Caetité 03,ACA-02-07,759560.0,8480930.0,...,UTM 23 S (MER -45),Tanque Novo VI Energias Renováveis S.A.,None,14460,BA,EOLCVBA032804-9-01,POINT (-42.59987 -13.72922),-42.599873,-13.729215,2023-06-12 11:32:39-03:00


## Visualização de valores extremos de potência nominal

In [10]:
gdf_total.sort_values(by="POT_MW", ascending=False)

,POT_MW,ALT_TOTAL,ALT_TORRE,DIAM_ROTOR,DATA_ATUALIZACAO,EOL_VERSAO_ID,NOME_EOL,DEN_AEG,X,Y,...,FUSO_AG,PROPRIETARIO,ORIGEM,OBJECTID,UF,CEG,geometry,longitude,latitude,DATA_ATUALIZACAO_FORMATADA
23171,6750.0000,196.27,196.27,172.54,1744724582000,NaN,Asa Branca III,AB-3-10,0.0,0.0,...,UTM 25 S (MER -33),None,None,23172,None,None,POINT (-35.87205 -5.21537),-35.872051,-5.215373,2025-04-15 10:43:02-03:00
15145,4200.0000,220.00,145.00,150.00,1690562954000,31444.0,São Manoel,SM-06,267283.0,6506212.0,...,UTM 22 S (MER -51),Empresa de Energia São Manoel S.A.,None,15146,RS,EOLCVRS034820-1-01,POINT (-53.45168 -31.55562),-53.451685,-31.555621,2023-07-28 13:49:14-03:00
15141,4200.0000,220.00,145.00,150.00,1690562954000,31444.0,São Manoel,SM-02,267643.0,6503440.0,...,UTM 22 S (MER -51),Empresa de Energia São Manoel S.A.,None,15142,RS,EOLCVRS034820-1-01,POINT (-53.44855 -31.58068),-53.448548,-31.580681,2023-07-28 13:49:14-03:00
15143,4200.0000,220.00,145.00,150.00,1690562954000,31444.0,São Manoel,SM-04,267343.0,6504888.0,...,UTM 22 S (MER -51),Empresa de Energia São Manoel S.A.,None,15144,RS,EOLCVRS034820-1-01,POINT (-53.45137 -31.56757),-53.451365,-31.567568,2023-07-28 13:49:14-03:00
15144,4200.0000,220.00,145.00,150.00,1690562954000,31444.0,São Manoel,SM-05,267065.0,6505455.0,...,UTM 22 S (MER -51),Empresa de Energia São Manoel S.A.,None,15145,RS,EOLCVRS034820-1-01,POINT (-53.45416 -31.5624),-53.454158,-31.562401,2023-07-28 13:49:14-03:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22977,0.0042,195.00,120.00,150.00,1710170491000,47192.0,Ventos de Santa Rosa 10,RSA-10-05,183477.0,8976610.0,...,UTM 24 S (MER -39),Dom Inocêncio Energias Renováveis S.A,None,22978,PI,EOLCVPI047192-5-01,POINT (-41.88055 -9.24655),-41.880552,-9.246554,2024-03-11 12:21:31-03:00
22978,0.0042,195.00,120.00,150.00,1710170491000,47197.0,Ventos de Santa Rosa 15,RSA-15-01,178086.0,8976624.0,...,UTM 24 S (MER -39),Dom Inocêncio Energias Renováveis S.A,None,22979,PI,EOLCVPI047197-6-01,POINT (-41.92957 -9.24603),-41.929569,-9.246031,2024-03-11 12:21:31-03:00
22979,0.0042,195.00,120.00,150.00,1710170491000,47195.0,Ventos de Santa Rosa 13,RSA-13-05,180941.0,8976744.0,...,UTM 24 S (MER -39),Dom Inocêncio Energias Renováveis S.A,None,22980,PI,EOLCVPI047195-0-01,POINT (-41.9036 -9.24516),-41.903601,-9.245158,2024-03-11 12:21:31-03:00
22980,0.0042,195.00,120.00,150.00,1710170491000,47192.0,Ventos de Santa Rosa 10,RSA-10-04,183645.0,8976805.0,...,UTM 24 S (MER -39),Dom Inocêncio Energias Renováveis S.A,None,22981,PI,EOLCVPI047192-5-01,POINT (-41.87901 -9.2448),-41.879010,-9.244805,2024-03-11 12:21:31-03:00


In [11]:
gdf_total.sort_values(by="POT_MW", ascending=True)

,POT_MW,ALT_TOTAL,ALT_TORRE,DIAM_ROTOR,DATA_ATUALIZACAO,EOL_VERSAO_ID,NOME_EOL,DEN_AEG,X,Y,...,FUSO_AG,PROPRIETARIO,ORIGEM,OBJECTID,UF,CEG,geometry,longitude,latitude,DATA_ATUALIZACAO_FORMATADA
23085,0.0042,195.00,120.00,150.00,1710170491000,47185.0,Ventos de Santa Rosa 02,RSA-02-10,192907.0,8989272.0,...,UTM 24 S (MER -39),Dom Inocêncio Energias Renováveis S.A,None,23086,PI,EOLCVPI047185-2-01,POINT (-41.79391 -9.13284),-41.793907,-9.132842,2024-03-11 12:21:31-03:00
23092,0.0042,195.00,120.00,150.00,1710170491000,47186.0,Ventos de Santa Rosa 03,RSA-03-09,194484.0,8990043.0,...,UTM 24 S (MER -39),Dom Inocêncio Energias Renováveis S.A,None,23093,PI,EOLCVPI047186-0-01,POINT (-41.77952 -9.12599),-41.779517,-9.125987,2024-03-11 12:21:31-03:00
23049,0.0042,195.00,120.00,150.00,1710170491000,49468.0,Ventos de Santa Rosa 06,RSA-06-01,189995.0,8983556.0,...,UTM 24 S (MER -39),Dom Inocêncio Energias Renováveis S.A,None,23050,PI,EOLCVPI049468-2-02,POINT (-41.82079 -9.18428),-41.820786,-9.184275,2024-03-11 12:21:31-03:00
23093,0.0042,195.00,120.00,150.00,1710170491000,47185.0,Ventos de Santa Rosa 02,RSA-02-07,192756.0,8990085.0,...,UTM 24 S (MER -39),Dom Inocêncio Energias Renováveis S.A,None,23094,PI,EOLCVPI047185-2-01,POINT (-41.79522 -9.12549),-41.795223,-9.125487,2024-03-11 12:21:31-03:00
22987,0.0042,195.00,120.00,150.00,1710170491000,47191.0,Ventos de Santa Rosa 09,RSA-09-12,184243.0,8977528.0,...,UTM 24 S (MER -39),Dom Inocêncio Energias Renováveis S.A,None,22988,PI,EOLCVPI047191-7-01,POINT (-41.87352 -9.23832),-41.873520,-9.238317,2024-03-11 12:21:31-03:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15152,4200.0000,220.00,145.00,150.00,1690562954000,31444.0,São Manoel,SM-06,267283.0,6506212.0,...,UTM 22 S (MER -51),Empresa de Energia São Manoel S.A.,None,15153,RS,EOLCVRS034820-1-01,POINT (-53.45168 -31.55562),-53.451685,-31.555621,2023-07-28 13:49:14-03:00
15153,4200.0000,220.00,145.00,150.00,1690562954000,31444.0,São Manoel,SM-07,268147.0,6506984.0,...,UTM 22 S (MER -51),Empresa de Energia São Manoel S.A.,None,15154,RS,EOLCVRS034820-1-01,POINT (-53.44241 -31.54884),-53.442408,-31.548836,2023-07-28 13:49:14-03:00
15149,4200.0000,220.00,145.00,150.00,1690562954000,31444.0,São Manoel,SM-03,267366.0,6504426.0,...,UTM 22 S (MER -51),Empresa de Energia São Manoel S.A.,None,15150,RS,EOLCVRS034820-1-01,POINT (-53.45123 -31.57174),-53.451232,-31.571737,2023-07-28 13:49:14-03:00
15144,4200.0000,220.00,145.00,150.00,1690562954000,31444.0,São Manoel,SM-05,267065.0,6505455.0,...,UTM 22 S (MER -51),Empresa de Energia São Manoel S.A.,None,15145,RS,EOLCVRS034820-1-01,POINT (-53.45416 -31.5624),-53.454158,-31.562401,2023-07-28 13:49:14-03:00


## Verificação da usina Serra de Gentio do Ouro XXIII

In [13]:
gdf_total.loc[gdf_total["NOME_EOL"].isin(["Serra de Gentio do Ouro XXIII", "Serra do Gentio do Ouro XXIII"])]

,POT_MW,ALT_TOTAL,ALT_TORRE,DIAM_ROTOR,DATA_ATUALIZACAO,EOL_VERSAO_ID,NOME_EOL,DEN_AEG,X,Y,...,FUSO_AG,PROPRIETARIO,ORIGEM,OBJECTID,UF,CEG,geometry,longitude,latitude,DATA_ATUALIZACAO_FORMATADA
10638,6.2000,200.0,115.0,170.0,1653929535000,55457.0,Serra do Gentio do Ouro XXIII,SGE-23-01,764188.000000,8.737531e+06,...,UTM 23 S (MER -45),Eólica Serra de Gentio do Ouro S.A.,None,10639,BA,EOLCVBA055457-0-01,POINT (-42.57892 -11.41045),-42.578923,-11.410455,2022-05-30 13:52:15-03:00
10639,6.2000,200.0,115.0,170.0,1653929535000,55457.0,Serra do Gentio do Ouro XXIII,SGE-23-02,764366.000000,8.737795e+06,...,UTM 23 S (MER -45),Eólica Serra de Gentio do Ouro S.A.,None,10640,BA,EOLCVBA055457-0-01,POINT (-42.57731 -11.40806),-42.577313,-11.408056,2022-05-30 13:52:15-03:00
10640,6.2000,200.0,115.0,170.0,1653929535000,55457.0,Serra do Gentio do Ouro XXIII,SGE-23-03,764522.000000,8.738085e+06,...,UTM 23 S (MER -45),Eólica Serra de Gentio do Ouro S.A.,None,10641,BA,EOLCVBA055457-0-01,POINT (-42.57591 -11.40542),-42.575907,-11.405424,2022-05-30 13:52:15-03:00
10641,6.2000,200.0,115.0,170.0,1653929535000,55457.0,Serra do Gentio do Ouro XXIII,SGE-23-04,764573.136000,8.738363e+06,...,UTM 23 S (MER -45),Eólica Serra de Gentio do Ouro S.A.,None,10642,BA,EOLCVBA055457-0-01,POINT (-42.57546 -11.40291),-42.575460,-11.402907,2022-05-30 13:52:15-03:00
23262,0.0248,200.0,NaN,170.0,1745490431000,55457.0,Serra de Gentio do Ouro XXIII,SGE-23-01,-42.578923,-1.141045e+01,...,UTM 23 S (MER -45),"Eolica Serra De Gentio Do Ouro Sa, Pec Energia...",None,23263,BA,EOLCVBA055457-0.01,POINT (-42.57892 -11.41045),-42.578923,-11.410455,2025-04-24 07:27:11-03:00
23263,0.0248,200.0,NaN,170.0,1745490431000,55457.0,Serra de Gentio do Ouro XXIII,SGE-23-02,-42.577313,-1.140806e+01,...,UTM 23 S (MER -45),"Eolica Serra De Gentio Do Ouro Sa, Pec Energia...",None,23264,BA,EOLCVBA055457-0.01,POINT (-42.57731 -11.40806),-42.577313,-11.408056,2025-04-24 07:27:11-03:00
23264,0.0248,200.0,NaN,170.0,1745490431000,55457.0,Serra de Gentio do Ouro XXIII,SGE-23-03,-42.575907,-1.140542e+01,...,UTM 23 S (MER -45),"Eolica Serra De Gentio Do Ouro Sa, Pec Energia...",None,23265,BA,EOLCVBA055457-0.01,POINT (-42.57591 -11.40542),-42.575907,-11.405424,2025-04-24 07:27:11-03:00
23265,0.0248,200.0,NaN,170.0,1745490785000,55457.0,Serra de Gentio do Ouro XXIII,SGE-23-04,-42.575460,-1.140291e+01,...,UTM 23 S (MER -45),"Eolica Serra De Gentio Do Ouro Sa, Pec Energia...",None,23266,BA,EOLCVBA055457-0.01,POINT (-42.57546 -11.40291),-42.575460,-11.402907,2025-04-24 07:33:05-03:00


## Verificação da usina Asa Branca III

In [25]:
gdf_total.loc[gdf_total["NOME_EOL"]=="Asa Branca III"]

,POT_MW,ALT_TOTAL,ALT_TORRE,DIAM_ROTOR,DATA_ATUALIZACAO,EOL_VERSAO_ID,NOME_EOL,DEN_AEG,X,Y,...,OPERACAO,FUSO_AG,PROPRIETARIO,ORIGEM,OBJECTID,UF,CEG,geometry,longitude,latitude
5821,2.7,150.00,89.00,122.00,1666625847000,30548.0,Asa Branca III,III-03,184880.0,9418696.0,...,Sim,UTM 25 S (MER -33),Nova Asa Branca III Energias Renováveis S.A.,None,5822,RN,EOLCVRN030548-0-01,POINT (-35.84203 -5.25304),-35.842029,-5.253036
6075,2.7,150.00,89.00,122.00,1666625847000,30548.0,Asa Branca III,III-01,185232.0,9419037.0,...,Sim,UTM 25 S (MER -33),Nova Asa Branca III Energias Renováveis S.A.,None,6076,RN,EOLCVRN030548-0-01,POINT (-35.83942 -5.24955),-35.839424,-5.249547
6247,2.7,150.00,89.00,122.00,1666625847000,30548.0,Asa Branca III,III-08,186880.0,9417212.0,...,Sim,UTM 25 S (MER -33),Nova Asa Branca III Energias Renováveis S.A.,None,6248,RN,EOLCVRN030548-0-01,POINT (-35.82464 -5.2661),-35.824644,-5.266104
6469,2.7,150.00,89.00,122.00,1666625847000,30548.0,Asa Branca III,III-05,184538.0,9418346.0,...,Sim,UTM 25 S (MER -33),Nova Asa Branca III Energias Renováveis S.A.,None,6470,RN,EOLCVRN030548-0-01,POINT (-35.84571 -5.25576),-35.845708,-5.255762
6619,2.7,150.00,89.00,122.00,1666625847000,30548.0,Asa Branca III,III-09,186683.0,9417019.0,...,Sim,UTM 25 S (MER -33),Nova Asa Branca III Energias Renováveis S.A.,None,6620,RN,EOLCVRN030548-0-01,POINT (-35.82643 -5.26784),-35.826427,-5.267840
7927,2.7,150.00,89.00,122.00,1666625847000,30548.0,Asa Branca III,III-07,184182.0,9418009.0,...,Sim,UTM 25 S (MER -33),Nova Asa Branca III Energias Renováveis S.A.,None,7928,RN,EOLCVRN030548-0-01,POINT (-35.84893 -5.25879),-35.848931,-5.258792
8055,2.7,150.00,89.00,122.00,1666625847000,30548.0,Asa Branca III,III-04,184708.0,9418527.0,...,Sim,UTM 25 S (MER -33),Nova Asa Branca III Energias Renováveis S.A.,None,8056,RN,EOLCVRN030548-0-01,POINT (-35.84417 -5.25413),-35.844168,-5.254134
8061,2.7,150.00,89.00,122.00,1666625847000,30548.0,Asa Branca III,III-10,186462.0,9416840.0,...,Sim,UTM 25 S (MER -33),Nova Asa Branca III Energias Renováveis S.A.,None,8062,RN,EOLCVRN030548-0-01,POINT (-35.82843 -5.26945),-35.828428,-5.269448
8491,2.7,150.00,89.00,122.00,1666625847000,30548.0,Asa Branca III,III-06,184357.0,9418181.0,...,Sim,UTM 25 S (MER -33),Nova Asa Branca III Energias Renováveis S.A.,None,8492,RN,EOLCVRN030548-0-01,POINT (-35.84735 -5.25725),-35.847346,-5.257245
8512,2.7,150.00,89.00,122.00,1666625847000,30548.0,Asa Branca III,III-02,185056.0,9418867.0,...,Sim,UTM 25 S (MER -33),Nova Asa Branca III Energias Renováveis S.A.,None,8513,RN,EOLCVRN030548-0-01,POINT (-35.84102 -5.25108),-35.841018,-5.251076
